In [1]:
import pandas as pd

import polars as pl
import polars.selectors as cs

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py

#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline

cf.go_offline()
cf.set_config_file(offline=True, world_readable=True)

In [3]:
from plots.Treemap import TreeMapRFM
from plots.Violin import violin_plot
from plots.Boxplot import box_plot

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [ ]:
COL_CUSTOMER_ID = 'Customer ID'
COL_ORDER_DATE = 'InvoiceDate'
COL_ORDER_ID = 'Invoice'
COL_VALUE = 'Sales'
COL_QUANTITY = 'Quantity'

https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci/

In [ ]:
df = pl.scan_csv(
    "F:/Data/datas/online_retail_II.csv", 
    dtypes = {'Customer ID': str, 'Invoice': str, 'InvoiceDate': pl.Datetime}
)

In [ ]:
df.describe()

### Data Cleaning and Preparation

#### Check and Drop Null Values

In [ ]:
df.describe()[1]

In [ ]:
df = df.drop_nulls()

####  Ignore duplicate rows

There are two possible reasons for duplicate rows.

There are two possible reasons for duplicate rows. 
1. The cashier might have scanned each product individually instead of scanning once and entering the quantity, resulting in multiple entries for each product on the invoice. 
2. The cashier might have accidentally scanned the same product twice. 

**Decision process**: The first scenario is more common, so we assume that the duplicate rows are not mistakes.<br>
**Action**: We ignore duplicate rows

#### Get only non-zero price value

In [ ]:
df = df.filter(pl.col('Price') != 0)

#### Drop rows that its StockCode column contains 'TEST' values

In [ ]:
df = df.filter(~pl.col('StockCode').str.contains('TEST'))

#### Drop the canceled invoice

In [ ]:
df = df.filter(~pl.col('Invoice').str.starts_with('C'))

#### Get Price Feature

In [ ]:
df = df.with_columns(Sales = pl.col('Price') * pl.col('Quantity'))

#### Drop Unnecessary features

In [ ]:
df = df.drop(['StockCode', 'Description', 'Price'])

In [ ]:
df.describe()

### Get RFM features

In [ ]:
df_rfm = df.group_by(COL_CUSTOMER_ID).agg(
    recency = pl.col(COL_ORDER_DATE).max(),
    frequency = pl.col(COL_ORDER_ID).n_unique(),
    monetary = pl.col(COL_VALUE).sum()
)
latest_date = df.select(pl.col(COL_ORDER_DATE).max()).collect().item()
df_rfm = df_rfm.with_columns(recency = (latest_date-pl.col('recency')).dt.total_days())

In [ ]:
df_rfm.describe()

#### Divide RFM values into 5 ranked groups

In [ ]:
max_f = df_rfm.select(pl.col('frequency').max()).collect().item()

# for frequency:
# (0-1]: {1}, (1-2]: {2}, (2-5]: {3, 4, 5}, (5-10]: {6, 7, 8, 9, 10}, (10-max_f]: {11, ...}
df_rfm = df_rfm.with_columns(
    r = pl.col('recency').qcut(quantiles=5, labels=['5', '4', '3', '2', '1']).cast(pl.UInt8),
    f = pl.col('frequency').cut(breaks=[1, 2, 5, 10], labels=['1', '2', '3', '4', '5']).cast(pl.UInt8),
    m = pl.col('monetary').qcut(quantiles=5, labels=['1', '2', '3', '4', '5']).cast(pl.UInt8)
)


In [ ]:
df1 = pl.DataFrame({"foo": [-2, -1, 0, 1, 2]})
df1.with_columns(pl.col("foo").cut([-1, 1], labels=["a", "b", "c"]).alias("cut"))

In [ ]:
# Polars notes that its qcut and cut methods are unstable, so this is the alternative
df_rfm = df_rfm.collect().to_pandas()
df_rfm['r'] = pd.qcut(df_rfm['recency'], q=5, labels=[5, 4, 3, 2, 1]).astype(int)
df_rfm['f'] = pd.cut(df_rfm['frequency'], [0, 1, 2, 5, 10, df_rfm['frequency'].max()], labels=[1, 2, 3, 4, 5]).astype(int)
df_rfm['m'] = pd.qcut(df_rfm['monetary'], q=5, labels=[1, 2, 3, 4, 5]).astype(int)
df_rfm = pl.from_pandas(df_rfm).lazy()

In [ ]:
df_rfm.select(pl.col('r', 'f', 'm').value_counts(sort=True)).collect()

### RFM Segment

#### RFM Score

In [ ]:
@pl.api.register_expr_namespace("RFM")
class RFM_score:
    def __init__(self, expr: pl.Expr):
        self._expr = expr
    
    def segment(self, regex_segments: dict) -> pl.Expr:
        for regex, segment in regex_segments.items():
            self._expr = self._expr.str.replace(regex, segment)
            
        return self._expr

In [ ]:
df_rfm = df_rfm.with_columns(
    rfm_score = (
        pl.col('r').cast(pl.String) + 
        ((pl.col('f') + pl.col('m')) / pl.lit(2)).round().cast(pl.Int8).cast(pl.String) 
    )
)

In [ ]:
seg_map = {
    r"[1-2][1-2]": "Hibernating",
    r"[1-2][3-4]": "At-Risk",
    r"[1-2]5": "Cannot Lose Them",
    r"3[1-2]": "About To Sleep",
    r"33": "Need Attention",
    r"[3-4][4-5]": "Loyal Customers",
    r"41": "Promising",
    r"51": "New Customers",
    r"[4-5][2-3]": "Potential Loyalists",
    r"5[4-5]": "Champions",
}

In [ ]:
df_rfm = df_rfm.with_columns(segment = pl.col('rfm_score').RFM.segment(seg_map))

In [ ]:
df_rfm.select(pl.col('segment').value_counts(sort=True)).collect()

#### Custom RFM Score

In [ ]:
@pl.api.register_lazyframe_namespace("RFM")
class RFM_custom_score:
    def __init__(self, ldf: pl.LazyFrame):
        self._ldf = ldf
    
    def _segment(self, r:tuple, f:tuple, m:tuple, label) -> pl.LazyFrame:
        return self._ldf.with_columns(
            pl.when(
                pl.col(self.col_r).is_between(r[0], r[1], 'both') & 
                pl.col(self.col_f).is_between(f[0], f[1], 'both') &
                pl.col(self.col_m).is_between(m[0], m[1], 'both')
            )
            .then(pl.lit(label))
            .otherwise(pl.col(self.col_name))
            .alias(self.col_name)
        )
    
    def segment(self, conditions: dict, col_r, col_f, col_m, col_name='segment'):
        """
        conditions: {'label': [(r_min, r_max), ( f_min, f_max), ( m_min, m_max)]}
        """
        self.col_r = col_r
        self.col_f = col_f
        self.col_m = col_m
        self.col_name = col_name
        
        self._ldf = self._ldf.with_columns(pl.lit(None).alias(col_name))
        
        for label, values in conditions.items():
            r, f, m = values[0], values[1], values[2]
            self._ldf = self._segment(r, f, m, label)
            
        return self._ldf

In [ ]:
segments = {
    'inactive': [(1, 1), (1, 5), (1, 5)],
    'at risk': [(2, 3), (1, 5), (1, 5)],
    'need high attention': [(2, 3), (2, 5), (2, 5)],
    'moderate': [(3, 5), (1, 5), (1, 5)],
    'potential loyalist': [(3, 5), (2, 5), (1, 3)],
    'potential high value': [(3, 5), (1, 3), (2, 5)],
    'new customers': [(5, 5), (1, 1), (1, 5)],
    'loyalist': [(4, 5), (4, 5), (2, 5)],
    'high value': [(4, 5), (2, 5), (4, 5)],    
    'champions': [(5, 5), (4, 5), (4, 5)]
}

In [ ]:
df_rfm = df_rfm.RFM.segment(segments, 'r', 'f', 'm')

In [ ]:
df_rfm.select(pl.col('segment').value_counts(sort=True)).collect()

### Data Analysis

#### TreeMap Summary Segment Report

In [ ]:
class TreeMapRFM:
    @staticmethod
    def get_group_segment(rfm_segment):
        rfm_segment_grouped = rfm_segment.groupby("segment").agg(
            size=("monetary", "size"),
            M_sum=("monetary", "sum"),
            M_mean=("monetary", "mean"),
            M_max=("monetary", "max"),
            M_min=("monetary", "min"),
            F_sum=("frequency", "sum"),
            F_mean=("frequency", "mean"),
            F_max=("frequency", "max"),
            F_min=("frequency", "min"),
            R_mean=("recency", "mean"),
            R_max=("recency", "max"),
            R_min=("recency", "min"),
        )
        rfm_segment_grouped["size_pctg"] = (
            (rfm_segment_grouped["size"] / rfm_segment.shape[0]) * 100
        ).round(2)
        rfm_segment_grouped["M_sum_pctg"] = (
            rfm_segment_grouped["M_sum"] / rfm_segment_grouped["M_sum"].sum()
        )
        rfm_segment_grouped["M_sum_pctg"] = (
            rfm_segment_grouped["M_sum_pctg"] * 100
        ).round(2)

        rfm_segment_grouped["F_sum_pctg"] = (
            rfm_segment_grouped["F_sum"] / rfm_segment_grouped["F_sum"].sum()
        )
        rfm_segment_grouped["F_sum_pctg"] = (
            rfm_segment_grouped["F_sum_pctg"] * 100
        ).round(2)

        return rfm_segment_grouped
    
    @staticmethod
    def plot_segment(rfm_segment, color_palette=None, auto_pctg=True, **layouts):  
        group_segment = TreeMapRFM.get_group_segment(rfm_segment).round(2)
        
        color_palette = (
            px.colors.qualitative.Pastel if not color_palette else color_palette
        )

        rfm_coordinates = {
            "Champions": [3, 5, 0.8, 1],
            "Loyal Customers": [3, 5, 0.4, 0.8],
            "Cannot Lose Them": [4, 5, 0, 0.4],
            "At-Risk": [2, 4, 0, 0.4],
            "Hibernating": [0, 2, 0, 0.4],
            "About To Sleep": [0, 2, 0.4, 0.6],
            "Promising": [0, 1, 0.6, 0.8],
            "New Customers": [0, 1, 0.8, 1],
            "Potential Loyalists": [1, 3, 0.6, 1],
            "Need Attention": [2, 3, 0.4, 0.6],
        }

        df_shp = pd.DataFrame(rfm_coordinates).T
        df_shp = df_shp.rename(columns={0: "y0", 1: "y1", 2: "x0", 3: "x1"})

        df_shp["fillcolor"] = color_palette[: df_shp.shape[0]]
        df_shp.loc[:, ["x0", "x1"]] = df_shp.loc[:, ["x0", "x1"]] * 5
        df_shp = df_shp.merge(group_segment, left_index=True, right_index=True)

        fig = go.Figure()
        for idx, val in df_shp.iterrows():
            x0, x1 = val["x0"], val["x1"]
            y0, y1 = val["y0"], val["y1"]

            texts = f"<b>{idx} | {val['size']} ({val['size_pctg']}%)</b><br>"

            text_m = "<b><br>Monetary</b><br>"
            text_total_m = (
                f"<b>Sum</b>: ${val['M_sum']} ({val['M_sum_pctg']}%)<br>"
            )
            text_max_m = f"<b>Max</b>: ${val['M_max']}<br>"
            text_mean_m = f"<b>Mean</b>: ${val['M_mean']}<br>"
            text_min_m = f"<b>Min</b>: ${val['M_min']}<br>"
            text_m += text_total_m + text_max_m + text_mean_m + text_min_m

            text_f = "<br><b>Frequency</b><br>"
            text_total_f = (
                f"<b>Sum</b>: {val['F_sum']} ({val['F_sum_pctg']}%)<br>"
            )
            text_max_f = f"<b>Max</b>: {val['F_max']}<br>"
            text_mean_f = f"<b>Mean</b>: {val['F_mean']}<br>"
            text_min_f = f"<b>Min</b>: {val['F_min']}<br>"
            text_f += text_total_f + text_max_f + text_mean_f + text_min_f

            text_r = "<br><b>Recency</b><br>"
            text_max_r = f"<b>Max</b>: {val['R_max']} days<br>"
            text_mean_r = f"<b>Mean</b>: {val['R_mean']} days<br>"
            text_min_r = f"<b>Min</b>: {val['R_min']} days<br>"
            text_r += text_max_r + text_mean_r + text_min_r

            texts += text_m + text_f + text_r

            fig.add_trace(
                go.Scatter(
                    x=[x0, x0, x1, x1, x0],
                    y=[y0, y1, y1, y0, y0],
                    fill="toself",
                    fillcolor=val["fillcolor"],
                    mode="lines",
                    #                 name=idx,
                    name="",
                    showlegend=False,
                    opacity=1,
                    line_width=1,
                    line_color="black",
                    text=texts,
                    hoveron="fills",
                    hoverlabel_bgcolor="#192655",
                    hoverlabel_align="left",
                    hoverlabel_namelength=-1,
                )
            )

        if auto_pctg is True:
            df_shp["text"] = df_shp.index.map(
                lambda x: f"<b>{x}<br>({df_shp.loc[x, 'size_pctg']}%)</b>"
            )

        fig.add_trace(
            go.Scatter(
                x=df_shp.loc[:, ["x0", "x1"]].mean(axis=1),
                y=df_shp.loc[:, ["y0", "y1"]].mean(axis=1),
                text=df_shp["text"],
                mode="text",
                showlegend=False,
                hoverinfo="skip",
            )
        )

        fig.update_layout(
            xaxis=dict(range=[0, 5], dtick=1, showgrid=False),
            yaxis=dict(range=[0, 5], showgrid=False),
            xaxis_title="Recency",
            yaxis_title="Frequency"
            if "rf_segment" in group_segment.columns
            else "Frequency & Monetary",
            margin={"l": 0, "r": 20, "b": 0, "t": 0},
            paper_bgcolor="rgba(0,0,0,0)",
            plot_bgcolor="rgba(0,0,0,0)",
        )

        fig.update_layout(**layouts)
        fig.update_traces(textfont_size=14, textfont_color="white")

        return fig

In [ ]:
df_rfm = df_rfm.collect().to_pandas()

In [ ]:
df_rfm_grouped = TreeMapRFM.get_group_segment(df_rfm).round(2)

In [ ]:
df_rfm_grouped.sort_values('M_sum')

In [ ]:
TreeMapRFM.plot_segment(df_rfm)

#### Distributions

##### Clean the outlier

In [ ]:
def get_rid_outlier(df, col, drop=False):
    df = df.copy()
    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3-Q1
    top_cap = Q3 + (IQR*1.5)
    bottom_cap = Q1 - (IQR*1.5)
    
    if drop:
        return df.loc[(df[col] >= bottom_cap) & (df[col] <= top_cap)]
    else:
        df.loc[df[col] < bottom_cap, col] = bottom_cap
        df.loc[df[col] > top_cap, col] = top_cap       
        
        return df

In [ ]:
df_rfm_no_outlier = df_rfm.copy()
df_rfm_no_outlier = get_rid_outlier(df_rfm_no_outlier, 'monetary')
df_rfm_no_outlier = get_rid_outlier(df_rfm_no_outlier, 'recency')
df_rfm_no_outlier = get_rid_outlier(df_rfm_no_outlier, 'frequency')

##### Box Plot

In [ ]:
def box_plot(rfm_segment, col_value="monetary", col_segment="segment"):
    df = rfm_segment.copy()
    segments = df.groupby(col_segment)[col_value].mean().sort_values().index
    colors = px.colors.sequential.Viridis[::-1][: len(segments)]
    datas = [df[df[col_segment] == segment][col_value].values for segment in segments]

    fig = go.Figure()
    for segment, data, color in zip(segments, datas, colors):
        fig.add_trace(go.Box(x=data, line_color=color, name=segment))

    fig.update_traces(orientation="h", boxpoints=False, boxmean=True, notched=True)
    fig.update_layout(xaxis_range=(0, datas[0]), template='plotly_white')

    return fig

In [ ]:
df_rfm_no_outlier['segment'].value_counts()

In [ ]:
fig_box = box_plot(df_rfm_no_outlier, col_value='monetary')

In [ ]:
fig_box.show()

##### Violin Plot

In [ ]:
def violin_plot(rfm_segment, col_value):
    df = rfm_segment.copy()
    segments = df.groupby("segment")[col_value].mean().sort_values().index
    datas = [df[df["segment"] == segment][col_value].values for segment in segments]
    colors = px.colors.sequential.Viridis[::-1][: len(segments)]

    fig = go.Figure()
    for segment, data, color in zip(segments, datas, colors):
        fig.add_trace(go.Violin(x=data, line_color=color, name=segment))

    fig.update_traces(
        orientation="h",
        side="positive",
        width=1,
        points=False,
        meanline_visible=True,
    )
    fig.update_layout(template='plotly_white')
    

    return fig

In [ ]:
fig_violin = violin_plot(df_rfm_no_outlier, 'recency')

In [ ]:
fig_violin